In [ ]:
import pickle
import gzip
import numpy as np

INPUT_DIR = '../input/'
OUTPUT_DIR = '../output/'
MAIN_DIR = 'C:/data/tf_speech/'
TRAIN_DIR = MAIN_DIR + 'train/'
TRAIN_MODIFIED_DIR = MAIN_DIR + 'train_modified/'
TRAIN_AUDIO_DIR = TRAIN_DIR + 'audio/'
BG_DIR = TRAIN_DIR + 'audio/_background_noise_/'
TEST_DIR = MAIN_DIR + 'test/audio/'
PREDS_DIR = '../output/preds/'
KFOLD_FILENAME = 'kfold_cache_4.pklz'

folds = pickle.load(gzip.open(TRAIN_MODIFIED_DIR + KFOLD_FILENAME, 'rb'))
for f in folds:
    print (len(f[0]), len(f[1]))


In [ ]:
folds[0][0][:10]

In [ ]:
import os
import glob
from utilities import *
from tqdm import tqdm, tqdm_notebook

def load_fold(fold):
    train_files = [f.split('\\')[-2] + f.split('\\')[-1] for f in fold[0]]
    val_files = [f.split('\\')[-2] + f.split('\\')[-1] for f in fold[1]]
    
    all_files = glob.glob(os.path.join(TRAIN_MODIFIED_AUDIO_DIR, '*/*wav'))

    train, val = [], []
    for fname in tqdm(all_files):
        splits = fname.split('\\')
        label, uid = splits[-2], splits[-1].split('_')[0]
        if label == '_background_noise_':
            label = 'silence'
        if label not in LABELS:
            label = 'unknown'

        label_id = NAME2ID[label]

        sample = (label_id, uid, fname)
        sample_id = splits[-2] + splits[-1]
        if sample_id in val_files:
            val.append(sample)
        elif sample_id in train_files:
            train.append(sample)
    print('There are {} train and {} val samples'.format(len(train), len(val)))

    return train, val

trainset, valset = load_fold(folds[0])
trainset, valset = load_fold(folds[1])
trainset, valset = load_fold(folds[2])
trainset, valset = load_fold(folds[3])

In [ ]:
trainset, valset = load_fold(folds[0])

In [ ]:
files = ['../output/preds/r50_f/fold_0/train.csv',
        '../output/preds/r50_f/fold_1/train.csv',
         '../output/preds/r50_f/fold_2/train.csv',
         '../output/preds/r50_f/fold_3/train.csv',
        ]

trainset0, valset0 = load_fold(folds[0])
trainset1, valset1 = load_fold(folds[1])
trainset2, valset2 = load_fold(folds[2])
trainset3, valset3 = load_fold(folds[3])

labels = []
for valset in [valset0, valset1, valset2, valset3]:
    labels.append([v[0] for v in valset])
    
    

In [10]:
labels = [item for sublist in labels for item in sublist]

In [11]:
len(labels)

72221

In [21]:
preds = []
for f in files:
    preds.append(pd.read_csv(f, index_col='fname').values)


In [22]:
preds = [item for sublist in preds for item in sublist]

In [23]:
preds = np.array(preds)

In [24]:
preds.shape

(72221, 12)

In [25]:
preds

array([[5.45341440e-07, 1.12733570e-02, 5.68119200e-09, ...,
        5.33063660e-04, 1.10157160e-06, 8.19366130e-04],
       [3.07502160e-05, 4.20786740e-02, 1.83860070e-07, ...,
        1.01446700e-02, 4.59887670e-05, 4.46214970e-02],
       [1.19450780e-04, 1.40774030e-02, 1.34401860e-03, ...,
        2.29340960e-02, 5.63298500e-03, 4.74986100e-01],
       ...,
       [2.74742580e-04, 8.54635440e-05, 8.02750400e-06, ...,
        6.07135100e-04, 9.98027400e-01, 5.09377200e-04],
       [1.49946390e-05, 3.99721700e-05, 1.28649670e-05, ...,
        3.90805070e-04, 9.98484700e-01, 8.63943800e-04],
       [4.91329900e-04, 1.03470775e-05, 3.34988200e-06, ...,
        5.97465260e-04, 5.93591750e-01, 3.91461520e-01]])

In [26]:
pred_labels = np.argmax(preds, axis=1)
pred_labels.shape

(72221,)

In [30]:
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix

conf = confusion_matrix(labels, pred_labels)
for l in LABELS:
    total = sum(conf[NAME2ID[l]])
    acc = 1.0 * conf[NAME2ID[l], NAME2ID[l]] / total
    print('{}, total: {}, accuracy: {}'.format(l, total,  acc))

yes, total: 2350, accuracy: 0.9829787234042553
no, total: 2347, accuracy: 0.9190455901150405
up, total: 2323, accuracy: 0.9130434782608695
down, total: 2329, accuracy: 0.8634607127522542
left, total: 2315, accuracy: 0.9520518358531318
right, total: 2320, accuracy: 0.8939655172413793
on, total: 2324, accuracy: 0.898881239242685
off, total: 2321, accuracy: 0.950021542438604
stop, total: 2347, accuracy: 0.9203238176395399
go, total: 2333, accuracy: 0.911701671667381
silence, total: 8347, accuracy: 0.9595064094884389
unknown, total: 40565, accuracy: 0.7977320350055467


### diff

In [72]:
import pandas as pd
from utilities import *
# diff -y --suppress-common-lines a b | grep '^' | wc -l
def diff_preds(f1, f2):
    p1 = pd.read_csv(f1).label.values
    p2 = pd.read_csv(f2).label.values
    
#     for i in range(len(p1)):
#         if p1[i] != p2[i]:
#             print (p1[i], p2[i])
    return sum(p1 != p2)

def diff_preds2(f1, f2):
    p1 = pd.read_csv(f1, index_col='fname').values
    fnames = pd.read_csv(f1, index_col='fname').index
    p1 = np.argmax(p1, axis=1)
    p2 = pd.read_csv(f2).label.values
    p1 = [ID2NAME[i] for i in p1]
    
    for i in range(len(p1)):
        if p1[i] != p2[i]:
            print (fnames[i], p1[i], p2[i])
    return sum(p1 != p2)

In [73]:
print(diff_preds2('../output/preds/inc_139_my.csv', '../output/subm_majorty.csv'))

clip_0003c7122.wav silence unknown
clip_0004c6707.wav silence go
clip_00123c913.wav silence unknown
clip_00139d600.wav off unknown
clip_00178d644.wav unknown no
clip_0018489ca.wav silence unknown
clip_0018ff8e9.wav silence unknown
clip_0023f6e02.wav no down
clip_0025e94de.wav unknown right
clip_0028ebc99.wav no unknown
clip_00291e83c.wav silence unknown
clip_002ddecb8.wav down no
clip_0030a3206.wav stop up
clip_0032d1f14.wav unknown left
clip_0033aedac.wav unknown go
clip_0034311e1.wav silence unknown
clip_003e10edb.wav off unknown
clip_004348a2b.wav silence unknown
clip_0044f8416.wav unknown silence
clip_004afbd03.wav silence unknown
clip_004cc2e62.wav silence unknown
clip_004d8390a.wav silence unknown
clip_005e71a11.wav silence unknown
clip_005f62fc3.wav unknown no
clip_0066e2e7e.wav silence unknown
clip_006b6248b.wav no go
clip_006c084dc.wav silence unknown
clip_0077e66de.wav silence unknown
clip_007b5eb43.wav unknown silence
clip_0084f4008.wav silence unknown
clip_008765519.wav lef

clip_08d929f77.wav up unknown
clip_08dd457d5.wav silence unknown
clip_08e131919.wav silence unknown
clip_08e7181c5.wav unknown on
clip_08ea78c79.wav silence unknown
clip_08ecb0694.wav unknown right
clip_08f1f8903.wav silence go
clip_08f9c5eaa.wav silence unknown
clip_08fda1c94.wav left unknown
clip_08ffaf14c.wav unknown yes
clip_08ffcce91.wav silence unknown
clip_090203b5f.wav silence unknown
clip_0902313e2.wav silence unknown
clip_0902f2d94.wav on unknown
clip_090be8d88.wav up off
clip_090ecd701.wav up unknown
clip_0920eaf9c.wav unknown on
clip_09217eabc.wav unknown silence
clip_092212922.wav right unknown
clip_0922268a6.wav left unknown
clip_092696da2.wav unknown right
clip_0934b1c55.wav unknown silence
clip_093b37eee.wav silence unknown
clip_093d29f52.wav unknown off
clip_093f53408.wav silence unknown
clip_09402c21d.wav unknown on
clip_0945cfcf3.wav silence unknown
clip_095765612.wav silence unknown
clip_09576cc90.wav off on
clip_096146e57.wav silence go
clip_0961bfbb1.wav unknown o

clip_165b633e0.wav silence unknown
clip_165dc2bdd.wav off up
clip_1662cbf56.wav silence yes
clip_16693cfa3.wav unknown no
clip_166e7941b.wav unknown silence
clip_166f2a6fe.wav unknown go
clip_16728db6c.wav down on
clip_16795ea97.wav silence up
clip_167bce5ce.wav unknown silence
clip_167de7570.wav silence unknown
clip_168302281.wav up unknown
clip_16892bbaa.wav silence unknown
clip_168974de0.wav silence unknown
clip_168975fee.wav unknown silence
clip_168cd4cb6.wav unknown up
clip_168e4321f.wav unknown no
clip_168f46b9a.wav silence unknown
clip_1690429e1.wav unknown silence
clip_1694ab520.wav unknown silence
clip_1696b0d72.wav unknown silence
clip_169709770.wav unknown down
clip_169871f0c.wav silence unknown
clip_16a7b62ca.wav unknown silence
clip_16b4cca47.wav silence unknown
clip_16b62e42c.wav silence unknown
clip_16bf8059c.wav silence unknown
clip_16c483209.wav unknown silence
clip_16c6199e3.wav silence unknown
clip_16c78d21d.wav silence unknown
clip_16caed099.wav silence up
clip_16cf

clip_25aba832a.wav unknown on
clip_25abdef6d.wav silence unknown
clip_25ae7b5e4.wav silence unknown
clip_25b2f3f35.wav unknown silence
clip_25b56ea63.wav silence unknown
clip_25c66f40d.wav silence go
clip_25d3a0f14.wav silence go
clip_25d485940.wav silence unknown
clip_25d5c2207.wav unknown on
clip_25da30cb1.wav silence unknown
clip_25e2547c0.wav silence unknown
clip_25e276873.wav unknown no
clip_25e2b0dd0.wav silence unknown
clip_25efce5b9.wav silence unknown
clip_25fe873d2.wav silence unknown
clip_25ffe6834.wav silence unknown
clip_260096916.wav silence unknown
clip_260281685.wav silence unknown
clip_26035fe20.wav silence unknown
clip_2605d109c.wav silence unknown
clip_26103a09d.wav silence unknown
clip_2612b86c3.wav silence unknown
clip_2619e0add.wav no unknown
clip_261c138bd.wav off unknown
clip_261cdb308.wav no go
clip_261ef6968.wav unknown silence
clip_26269066c.wav unknown silence
clip_2626ee4c4.wav silence unknown
clip_262741cec.wav stop up
clip_262798d55.wav unknown silence
cl

clip_32639a813.wav silence unknown
clip_327135608.wav unknown right
clip_3271f4549.wav unknown down
clip_327382717.wav left no
clip_327a1b8c0.wav unknown on
clip_327f5eb33.wav unknown right
clip_3299ca265.wav silence unknown
clip_32a0af359.wav on off
clip_32a4fa0fe.wav unknown on
clip_32acf5a5b.wav silence unknown
clip_32aef25d2.wav silence unknown
clip_32af5bd35.wav silence unknown
clip_32af6df38.wav go on
clip_32afc60d5.wav silence unknown
clip_32b2b54ad.wav silence unknown
clip_32b2b8312.wav on down
clip_32b7007e7.wav silence unknown
clip_32b97dc20.wav unknown silence
clip_32b9c61ff.wav no unknown
clip_32ba7ee75.wav unknown silence
clip_32c4c1e6f.wav unknown left
clip_32c8bcd76.wav unknown silence
clip_32cad68fd.wav unknown silence
clip_32d75e18f.wav unknown stop
clip_32d8ddf18.wav unknown silence
clip_32d9dbbff.wav silence unknown
clip_32dbcc6b7.wav silence unknown
clip_32dc23ae3.wav silence unknown
clip_32dc9e3cf.wav unknown on
clip_32ddf13c9.wav off up
clip_32de1118c.wav unknown 

...

clip_c52294b2a.wav unknown silence
clip_c5316bbe8.wav silence unknown
clip_c5352913c.wav unknown silence
clip_c53795a59.wav silence unknown
clip_c5393e864.wav up off
clip_c53aaf251.wav silence unknown
clip_c53f6ab8f.wav silence unknown
clip_c54cf94c2.wav off up
clip_c54d762d2.wav silence unknown
clip_c54fa779f.wav unknown go
clip_c55334267.wav silence unknown
clip_c55b0ba0b.wav unknown down
clip_c560fcd3d.wav off on
clip_c5683bd3a.wav silence unknown
clip_c56959f1b.wav silence unknown
clip_c56cd69ca.wav unknown on
clip_c56cdbc97.wav silence unknown
clip_c56e39ada.wav left unknown
clip_c56ee37cd.wav off on
clip_c575531e9.wav off unknown
clip_c57e8291d.wav unknown silence
clip_c57ef2466.wav silence unknown
clip_c583ec92e.wav go down
clip_c58454c23.wav unknown go
clip_c5884f427.wav silence unknown
clip_c58dfa1de.wav silence unknown
clip_c596ceb96.wav unknown on
clip_c59a32514.wav silence unknown
clip_c59b482e2.wav silence unknown
clip_c59d5c6b0.wav silence unknown
clip_c59f288c7.wav unkno

clip_d37b3f501.wav unknown silence
clip_d37f50b1a.wav off up
clip_d37f6a32c.wav unknown no
clip_d38310438.wav unknown on
clip_d38aa0dcb.wav silence unknown
clip_d38d39408.wav unknown no
clip_d38e6b885.wav right unknown
clip_d39160e55.wav unknown silence
clip_d39db67d5.wav silence unknown
clip_d39e727b8.wav on unknown
clip_d3a38d4a7.wav silence unknown
clip_d3ab08948.wav silence unknown
clip_d3abaa454.wav unknown on
clip_d3c16c7e1.wav silence unknown
clip_d3c436972.wav up unknown
clip_d3c92e569.wav unknown down
clip_d3c93ee25.wav silence unknown
clip_d3ce45a04.wav silence unknown
clip_d3d0ea8f4.wav off unknown
clip_d3d5f55c9.wav silence go
clip_d3e1cdf48.wav unknown down
clip_d3e93c1e7.wav unknown silence
clip_d3ec2cb90.wav silence unknown
clip_d3f074ad6.wav unknown silence
clip_d3fb9a8e2.wav unknown silence
clip_d4041ac38.wav silence unknown
clip_d40614abd.wav unknown silence
clip_d40fc7efe.wav down yes
clip_d41048ea9.wav unknown no
clip_d417b280a.wav unknown up
clip_d41c8b8cb.wav yes 

clip_de8c10070.wav silence unknown
clip_de8cd6751.wav silence unknown
clip_de927c6cb.wav silence unknown
clip_de9697919.wav unknown go
clip_de99000b5.wav silence unknown
clip_de9abefd1.wav left up
clip_de9dc3ad9.wav silence unknown
clip_de9f3cf23.wav no unknown
clip_dea0a7719.wav silence unknown
clip_dea75a9e8.wav silence unknown
clip_deab98352.wav stop unknown
clip_deb07edb6.wav yes unknown
clip_deb6a02aa.wav unknown silence
clip_deb98cc1c.wav silence unknown
clip_decc340ec.wav unknown silence
clip_decff9a08.wav silence unknown
clip_ded3c3157.wav silence unknown
clip_ded555e9c.wav silence unknown
clip_dedbe563e.wav off on
clip_dee0d15ec.wav silence unknown
clip_dee1dcf89.wav silence unknown
clip_dee70377a.wav no go
clip_dee76dfb9.wav silence unknown
clip_dee972582.wav unknown silence
clip_deeb2e92c.wav off on
clip_def303c87.wav silence unknown
clip_def82bf3c.wav unknown left
clip_def94065d.wav silence unknown
clip_df01f9d60.wav silence unknown
clip_df03843f4.wav unknown silence
clip_d

clip_ec51c5aaa.wav unknown silence
clip_ec5d5f9e2.wav silence unknown
clip_ec5f5388a.wav unknown silence
clip_ec677a12e.wav unknown silence
clip_ec6a3f9f5.wav silence unknown
clip_ec6d62edb.wav no unknown
clip_ec73dd70e.wav silence unknown
clip_ec7803012.wav unknown go
clip_ec8f23c37.wav unknown go
clip_ec90409ed.wav silence unknown
clip_ec94e65bb.wav unknown no
clip_ec9eaf1e8.wav unknown silence
clip_ecac3e2c5.wav silence down
clip_ecacedf5f.wav unknown up
clip_ecb36ccab.wav silence unknown
clip_ecb5eb5fc.wav unknown go
clip_ecbcbc22c.wav unknown yes
clip_eccddd8a9.wav silence unknown
clip_ecd093864.wav silence unknown
clip_ecd2c93d8.wav unknown left
clip_ecd45008f.wav silence unknown
clip_ecd54a0d8.wav unknown silence
clip_ecd6b0422.wav unknown left
clip_ecdaed79b.wav silence unknown
clip_ecdc84a90.wav silence unknown
clip_ece5d0928.wav left right
clip_ece659094.wav unknown off
clip_eced4b15c.wav silence down
clip_ecee91eed.wav silence unknown
clip_ecf0832f4.wav silence unknown
clip_

clip_fae7334aa.wav silence unknown
clip_fae81861e.wav unknown silence
clip_faf1ad3c8.wav go stop
clip_faf1f98b4.wav unknown on
clip_faf313839.wav unknown silence
clip_fafce100a.wav silence unknown
clip_fafe9b84b.wav silence unknown
clip_fafeb5384.wav unknown on
clip_fb037e76c.wav unknown silence
clip_fb0777ecc.wav silence unknown
clip_fb0a46c91.wav silence unknown
clip_fb0cf4ffb.wav silence unknown
clip_fb1488e62.wav down unknown
clip_fb1c4f6e1.wav silence unknown
clip_fb27f2249.wav silence unknown
clip_fb28ad43c.wav up go
clip_fb2a604c0.wav silence unknown
clip_fb2c807e7.wav silence unknown
clip_fb306e8b0.wav silence unknown
clip_fb3882d33.wav unknown silence
clip_fb3942bfb.wav go down
clip_fb3f36788.wav unknown silence
clip_fb42f5e4d.wav unknown right
clip_fb49cfbe5.wav unknown silence
clip_fb4b955d8.wav no down
clip_fb5290bf8.wav unknown go
clip_fb52e1590.wav silence unknown
clip_fb53239a9.wav silence unknown
clip_fb57f449a.wav unknown on
clip_fb5c1736a.wav up stop
clip_fb5dbbf59.wa

In [161]:
from collections import Counter

p = pd.read_csv('../output/preds/mean_[v+r+i+x].csv', index_col='fname').values
p = np.argmax(p, axis=1)
p = [ID2NAME[i] for i in p]

best = pd.read_csv('../output/best.csv').label.values

count_p = Counter(list(p))
count_best = Counter(list(best))

# print(count_p)

# print(count_best)
# print(list(count_best.keys()))
# print(np.array(list(count_best.values())) - np.array(list(count_p1.values())))

keys = list(count_best.keys())
for k in keys:
    print('{}: {}'.format(k, count_best[k] - count_p[k]))
    
print(sum(p != best))

silence: -573
stop: 65
go: 506
no: 396
yes: 71
up: 325
down: 84
left: 183
on: 402
unknown: -1880
off: 204
right: 217
9082


In [ ]:
silence: 1797
stop: -70
go: 385
no: 119
yes: 6
up: -8
down: 272
left: -277
on: 312
unknown: -2786
off: 29
right: 221

In [117]:
from predict import make_submission

p1 = pd.read_csv('../output/preds/xcep_my_my_n.csv', index_col='fname').values
p2 = pd.read_csv('../output/preds/inc_139_my_my_n.csv', index_col='fname').values
p = np.argmax(0.5*p1 + 0.5*p2, axis=1)

make_submission(pd.read_csv('../output/preds/inc_139_my_my_n.csv').fname.values, p, '[i+x]_my_my_n')

In [113]:
preds = pd.read_csv('../output/preds/inc_139_my_n.csv', index_col='fname').values
pred_classes = np.argmax(preds, axis=1)

cond = np.logical_and(pred_classes == 10, preds[:,11] > 0.35)
preds[np.where(cond)[0], 11] = 1.0

pred_classes = np.argmax(preds, axis=1)
p = [ID2NAME[i] for i in pred_classes]

best = pd.read_csv('../output/best.csv').label.values
print(sum(p != best))

9479


new -> best: 12936
new -> old: 13912
old -> best: 10483

best - new:
unknown down stop go  left  on right  no silence up  off   yes
-639    132 -144  468 -964  543 -124  290  342   348 -230  -22

In [98]:
list(count_best)

['yes',
 'off',
 'on',
 'stop',
 'go',
 'right',
 'left',
 'down',
 'unknown',
 'silence',
 'no',
 'up']

In [ ]:
print(diff_preds2('../output/preds/r50_f.csv', '../output/subm_majorty.csv'))
print(diff_preds('../output/resnet50_224.csv', '../output/subm_majorty.csv'))
print(diff_preds2('../output/preds/r50_f.csv', '../output/resnet50_224.csv', ))

In [3]:
print(diff_preds('../output/r50_199_f.csv', '../output/subm_majorty.csv'))
print(diff_preds('../output/r50_199_f.csv', '../output/subm_majorty.csv'))

16653
16653


In [4]:
print(diff_preds2('../output/preds/r50_199_f.csv', '../output/subm_majorty.csv'))
print(diff_preds('../output/resnet50_fixed.csv', '../output/subm_majorty.csv'))
print(diff_preds2('../output/preds/r50_199_f.csv', '../output/resnet50_fixed.csv'))

15140
9307
14555


In [12]:
print(diff_preds('../output/inc_139.csv', '../output/subm_majorty.csv'))
print(diff_preds('../output/incres_140.csv', '../output/subm_majorty.csv'))
print(diff_preds('../output/inc_139.csv', '../output/incres_140.csv'))

17688
9458
17625


## diff for 5 models


In [41]:
files = ['../output/preds/resnet50_fixed.csv', '../output/preds/resnet50_224.csv', '../output/preds/inception.csv',
        '../output/preds/xception.csv', '../output/preds/vgg16_256_128.csv']

preds = []
for f in files:
    preds.append(pd.read_csv(f, header=None).values)
preds = np.array(preds)
print(preds.shape)

mean_preds_old = np.mean(preds, axis=0)
mean_labels_old = np.argmax(mean_preds, axis=1)

p_best = pd.read_csv('../output/subm_majorty.csv').label.values
p_best = np.array([NAME2ID[n] for n in p_best])
print(p_best.shape)
sum(mean_labels_old != p_best)

(5, 158538, 12)
(158538,)


7836

In [44]:
dirs = ['r50_199', 'r50_244', 'xcep', 'inc_139', 'v16']

preds = []
for d in dirs:
    files = ['../output/preds/{}/fold_{}/test.csv'.format(d, i) for i in range(4)]
    cur_preds = []
    for f in files:
        print(f)
        cur_preds.append(pd.read_csv(f, index_col='fname').values)
    cur_preds = np.mean(cur_preds, axis=0)
    preds.append(cur_preds)

mean_preds = np.mean(preds, axis=0)
mean_labels = np.argmax(mean_preds, axis=1)

p_best = pd.read_csv('../output/subm_majorty.csv').label.values
p_best = [NAME2ID[n] for n in p_best]
sum(mean_labels != p_best)

../output/preds/r50_199/fold_0/test.csv
../output/preds/r50_199/fold_1/test.csv
../output/preds/r50_199/fold_2/test.csv
../output/preds/r50_199/fold_3/test.csv
../output/preds/r50_244/fold_0/test.csv
../output/preds/r50_244/fold_1/test.csv
../output/preds/r50_244/fold_2/test.csv
../output/preds/r50_244/fold_3/test.csv
../output/preds/inc_139/fold_0/test.csv
../output/preds/inc_139/fold_1/test.csv
../output/preds/inc_139/fold_2/test.csv
../output/preds/inc_139/fold_3/test.csv
../output/preds/v16/fold_0/test.csv
../output/preds/v16/fold_1/test.csv
../output/preds/v16/fold_2/test.csv
../output/preds/v16/fold_3/test.csv


14882

In [43]:
sum(mean_labels_old != mean_labels)

13555